# Evaluate no-show percentages during pilot

This notebook is used for evaluating the no-show pilot by looking at the no-show percentage

In [ ]:
import os
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from IPython.display import display as ipydisplay
from ipywidgets import interact  # type: ignore
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from noshow.model.predict import create_prediction
from noshow.preprocessing.load_data import (
    load_appointment_csv,
    process_appointments,
    process_postal_codes,
)


In [ ]:
load_dotenv("../.env")


# Global and env variables
db_user = os.environ["DB_USER"]
db_passwd = os.environ["DB_PASSWD"]
db_host = os.environ["DB_HOST"]
db_port = os.environ["DB_PORT"]
db_database = os.environ["DB_DATABASE"]

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_pilot.csv")
appointments_df = process_appointments(appointments_df).sort_index()
appointments_df["pilot"] = "Geen pilot"
appointments_df.loc[(slice(None), slice("2023-10-05", "2023-11-01")), "pilot"] = "pilot"
appointments_df.shape

In [ ]:
CONNECTSTRING = (
    rf"mssql+pymssql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_database}"
)
engine = create_engine(CONNECTSTRING)
session_object = sessionmaker(bind=engine)

In [ ]:
call_response = pd.read_sql_table("apicallresponse", engine, schema="noshow")
prediction = pd.read_sql_table("apiprediction", engine, schema="noshow")

prediction_response = prediction.merge(
    call_response, left_on="id", right_on="prediction_id", how="inner"
)
prediction_response

## No show per month

In [ ]:
appointments_df["month"] = appointments_df.index.get_level_values("start").to_period(
    "M"
)
appointments_df.groupby("month")["no_show"].value_counts(True).unstack()[
    "no_show"
].plot.bar(figsize=(20, 6))

### Per clinic

In [ ]:
for agenda in appointments_df["hoofdagenda"].unique():
    (
        appointments_df.loc[appointments_df["hoofdagenda"] == agenda]
        .groupby("month")["no_show"]
        .value_counts(True)
        .unstack()["no_show"]
        .plot.bar(figsize=(20, 6), title=agenda)
    )
    plt.show()

## No show aggregated on month

In [ ]:
appointments_df["month"] = appointments_df.index.get_level_values("start").month
tmp_df = (
    appointments_df.groupby(["month", "pilot"])["no_show"]
    .value_counts(True)
    .unstack(["no_show", "pilot"])["no_show"]
)
tmp_df.loc[11, "pilot"] = None
print(tmp_df)
tmp_df.plot.bar(figsize=(15, 6))
plt.xlabel("")
plt.title("Gem. no-show percentage per maand sinds 2015")
plt.show()

### Per clinic

In [ ]:
for agenda in appointments_df["hoofdagenda"].unique():
    tmp_df = (
        appointments_df.loc[appointments_df["hoofdagenda"] == agenda]
        .groupby(["month", "pilot"])["no_show"]
        .value_counts(True)
        .unstack(["no_show", "pilot"])["no_show"]
    )
    tmp_df.loc[11, "pilot"] = None
    tmp_df.plot.bar(figsize=(15, 6))
    plt.xlabel("")
    plt.title(agenda)
    plt.show()

## No-show percentage per prediction bin

In [ ]:
with Path("../output/models/no_show_model_cv.pickle").open("rb") as f:
    model = pickle.load(f)

all_postalcodes = process_postal_codes("../data/raw/NL.txt")
predictions_df = create_prediction(model, appointments_df, all_postalcodes)

In [ ]:
# Control group is Oct 2015 - 2022
predictions_df = predictions_df.loc[
    predictions_df.index.get_level_values("start").month == 10
]

In [ ]:
total_appointments = appointments_df.join(predictions_df, how="inner")
total_appointments["predict_bin"] = pd.qcut(total_appointments["prediction"], 12)

In [ ]:
total_appointments.groupby(["pilot", "predict_bin"])["no_show"].value_counts(
    True
).unstack(["no_show", "pilot"])["no_show"].plot.bar()

In [ ]:
total_appointments["noshow_num"] = 0
total_appointments.loc[total_appointments["no_show"] == "no_show", "noshow_num"] = 1
total_appointments_plot = total_appointments.groupby(["pilot", "predict_bin"])[
    "noshow_num"
].agg(["mean", "std", "size"])
total_appointments_plot

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
total_appointments[["prediction", "pilot"]].plot.hist(by="pilot", bins=100, ax=ax)
fig.show()

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


@interact
def no_show_perc_plot(
    years=years,
    show_all=False,
    only_called=False,
):
    """Plot No-Show percentages over selected years"""
    if show_all:
        total_appointments_selection = total_appointments
    else:
        year_selection = [years, 2023]
        total_appointments_selection = total_appointments[
            total_appointments.index.get_level_values("start").year.isin(year_selection)
        ]

    if only_called:
        total_appointments_selection = total_appointments_selection.loc[
            (total_appointments_selection["pilot"] == "Geen pilot")
            | total_appointments_selection["APP_ID"].isin(
                prediction_response["prediction_id"].astype(int)
            )
        ]

    plt.subplots(figsize=(15, 6))
    sns.barplot(
        data=total_appointments_selection,
        x="predict_bin",
        y="noshow_num",
        hue="pilot",
        hue_order=["Geen pilot", "pilot"],
    )
    plt.title("No-Show percentage per risico-categorie")
    plt.xlabel("Risico-categorieen")
    plt.ylabel("Percentage No-Show")
    plt.show()

    total_appointments_plot = total_appointments_selection.groupby(
        ["pilot", "predict_bin"]
    )["noshow_num"].agg(["mean", "std", "size"])
    ipydisplay(total_appointments_plot)

In [ ]:
prediction_response.columns

In [ ]:
prediction_response["id_x"] = prediction_response["id_x"].astype("Int64")
tmp = total_appointments.merge(prediction_response, left_on="APP_ID", right_on="id_x")
tmp = tmp[tmp["call_status"] == "Gebeld"]
tmp["y"] = tmp["call_outcome"] == "Verzet/Geannuleerd"
tmp